In [1]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image

# Directorio donde están las imágenes organizadas por carpetas (una por cada set)
data_dir = "../TRABAJO/FOTOS"

# Transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar todas las imágenes
    transforms.RandomHorizontalFlip(),  # Aumento de datos: reflejo horizontal
    transforms.RandomRotation(15),  # Aumento de datos: rotación aleatoria
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Ajuste de color
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize([0.5], [0.5])  # Normalizar valores de píxeles
])

# Cargar dataset con ImageFolder (cada carpeta es una clase)
dataset = ImageFolder(root=data_dir, transform=transform)

# Dividir en entrenamiento (80%), validación (10%) y prueba (10%)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Crear DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Dataset preprocesado y dividido:")
print(f"- Entrenamiento: {len(train_dataset)} imágenes")
print(f"- Validación: {len(val_dataset)} imágenes")
print(f"- Prueba: {len(test_dataset)} imágenes")


Dataset preprocesado y dividido:
- Entrenamiento: 1062 imágenes
- Validación: 132 imágenes
- Prueba: 134 imágenes


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Comprobar si hay GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar el modelo preentrenado EfficientNetB0
model = models.efficientnet_b0(pretrained=True)

# Modificar la última capa para clasificar nuestros sets de LEGO
num_features = model.classifier[1].in_features
num_classes = len(dataset.classes)  # Número de sets de LEGO
model.classifier[1] = nn.Linear(num_features, num_classes)

# Enviar el modelo a GPU si está disponible
model = model.to(device)

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_acc = 100 * correct / total
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    # Evaluación en conjunto de validación
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    
    val_acc = 100 * correct_val / total_val

    print(f"Época {epoch+1}/{num_epochs} - Pérdida: {running_loss/len(train_loader):.4f} - "
          f"Acc. Entrenamiento: {train_acc:.2f}% - Acc. Validación: {val_acc:.2f}%")

# Guardar el modelo entrenado
torch.save(model.state_dict(), "../04_Extra/ID/modelo_lego.pth")
print("Entrenamiento finalizado. Modelo guardado.")


/Users/luismgl/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/luismgl/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /Users/luismgl/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:32<00:00, 669kB/s] 


Época 1/20 - Pérdida: 3.0010 - Acc. Entrenamiento: 29.94% - Acc. Validación: 61.36%
Época 2/20 - Pérdida: 1.1246 - Acc. Entrenamiento: 73.35% - Acc. Validación: 69.70%
Época 3/20 - Pérdida: 0.5109 - Acc. Entrenamiento: 87.76% - Acc. Validación: 80.30%
Época 4/20 - Pérdida: 0.3732 - Acc. Entrenamiento: 90.77% - Acc. Validación: 76.52%
Época 5/20 - Pérdida: 0.2413 - Acc. Entrenamiento: 94.54% - Acc. Validación: 81.06%
Época 6/20 - Pérdida: 0.2301 - Acc. Entrenamiento: 94.16% - Acc. Validación: 81.06%
Época 7/20 - Pérdida: 0.1565 - Acc. Entrenamiento: 96.05% - Acc. Validación: 81.06%
Época 8/20 - Pérdida: 0.0919 - Acc. Entrenamiento: 98.31% - Acc. Validación: 81.06%
Época 9/20 - Pérdida: 0.0751 - Acc. Entrenamiento: 98.40% - Acc. Validación: 84.85%
Época 10/20 - Pérdida: 0.0764 - Acc. Entrenamiento: 98.21% - Acc. Validación: 82.58%
Época 11/20 - Pérdida: 0.1471 - Acc. Entrenamiento: 96.42% - Acc. Validación: 78.79%
Época 12/20 - Pérdida: 0.1310 - Acc. Entrenamiento: 97.27% - Acc. Validaci

In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import numpy as np
import json
from collections import Counter

# Configuración del dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Directorio del dataset de fotos 
data_dir = "../TRABAJO/FOTOS"

# Data Augmentation mejorado
transform = transforms.Compose([
    transforms.Resize((400, 400)),
    transforms.RandomResizedCrop(224, scale=(0.7, 1.3)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.8, contrast=0.8, saturation=0.8, hue=0.4),
    transforms.RandomAffine(degrees=30, translate=(0.3, 0.3)),
    transforms.RandomPerspective(distortion_scale=0.5, p=0.7),
    transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 3.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Cargamos el dataset
dataset = ImageFolder(root=data_dir, transform=transform)
num_classes = len(dataset.classes)

# Guardar el mapeo de clases a nombres de sets
class_to_idx = dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

# Guardar el mapeo en un archivo
with open("idx_to_class.json", "w") as f:
    json.dump(idx_to_class, f)

# División correcta del dataset
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size  # Asegurar suma exacta

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Depuración: Verificar tamaños
print(f"Total dataset size: {len(dataset)}")
print(f"Train set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

# Contar clases en el conjunto de entrenamiento
train_targets = [dataset.targets[i] for i in train_dataset.indices]
class_counts = Counter(train_targets)
class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}

# Crear lista de pesos basada en el conjunto de entrenamiento
samples_weights = [class_weights[t] for t in train_targets]

# Aplicar sampler al DataLoader de entrenamiento
sampler = WeightedRandomSampler(samples_weights, num_samples=len(samples_weights), replacement=True)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Cargamos un modelo más potente (EfficientNetB3)
from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights
model = efficientnet_b3(weights=EfficientNet_B3_Weights.DEFAULT)
num_features = model.classifier[1].in_features

# Modificar la arquitectura del clasificador con más Dropout
model.classifier = nn.Sequential(
    nn.Dropout(0.6),
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Linear(512, num_classes)
)

model = model.to(device)

# Ajustamos el Optimizer y Learning Rate
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=5e-4)
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)
criterion = nn.CrossEntropyLoss()
early_stopping_patience = 4
best_val_loss = np.inf
epochs_no_improve = 0

# Implementar CutMix

def cutmix_data(x, y, alpha=1.0):
    lam = np.random.beta(alpha, alpha)
    rand_index = torch.randperm(x.size()[0]).to(x.device)
    target_a, target_b = y, y[rand_index]
    return x, target_a, target_b, lam

# Entrenamiento del modelo
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    if epoch == 14:
        print("🔄 Reducción de tasa de aprendizaje a 50%...")
        for param_group in optimizer.param_groups:
            param_group["lr"] *= 0.5

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        if epoch <= 14 and np.random.rand() > 0.5:
            images, targets_a, targets_b, lam = cutmix_data(images, labels)
            outputs = model(images)
            loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
        running_loss += loss.item()

    train_acc = 100 * correct_train / total_train
    avg_train_loss = running_loss / len(train_loader)

    print(f"Época {epoch+1}/{num_epochs} - "
          f"Pérdida Entrenamiento: {avg_train_loss:.4f} - Acc. Entrenamiento: {train_acc:.2f}%")

print("🎉 Entrenamiento finalizado. Modelo guardado como 'modelo_lego_final.pth'.")


Usando dispositivo: cpu
Total dataset size: 1328
Train set size: 1062
Validation set size: 132
Test set size: 134


KeyboardInterrupt: 